In [1]:
#Prints **all** console output, not just last item in cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**Notebook author:** emeinhardt@ucsd.edu

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Step-0:-Check-for-foundational-files" data-toc-modified-id="Step-0:-Check-for-foundational-files-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Step 0: Check for foundational files</a></span><ul class="toc-item"><li><span><a href="#Step-0a:-Check-for-gating-data" data-toc-modified-id="Step-0a:-Check-for-gating-data-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Step 0a: Check for gating data</a></span></li><li><span><a href="#Step-0b:-Check-for-transcribed-lexicons" data-toc-modified-id="Step-0b:-Check-for-transcribed-lexicons-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Step 0b: Check for transcribed lexicons</a></span></li><li><span><a href="#Step-0c:-Check-for-n-gram-contexts" data-toc-modified-id="Step-0c:-Check-for-n-gram-contexts-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Step 0c: Check for n-gram contexts</a></span></li><li><span><a href="#Step-0d:-Check-for-language-model(s)" data-toc-modified-id="Step-0d:-Check-for-language-model(s)-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Step 0d: Check for language model(s)</a></span></li></ul></li><li><span><a href="#Step-1:-Segment-inventory-alignment" data-toc-modified-id="Step-1:-Segment-inventory-alignment-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Step 1: Segment inventory alignment</a></span><ul class="toc-item"><li><span><a href="#Step-1a:-Define-inventory-alignment-projections" data-toc-modified-id="Step-1a:-Define-inventory-alignment-projections-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Step 1a: Define inventory alignment projections</a></span></li><li><span><a href="#Step-1b:-Apply-inventory-alignment-projections" data-toc-modified-id="Step-1b:-Apply-inventory-alignment-projections-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Step 1b: Apply inventory alignment projections</a></span><ul class="toc-item"><li><span><a href="#Check-for-projection-definitions" data-toc-modified-id="Check-for-projection-definitions-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>Check for projection definitions</a></span></li><li><span><a href="#How-are-inventory-alignment-projections-actually-applied?" data-toc-modified-id="How-are-inventory-alignment-projections-actually-applied?-4.2.2"><span class="toc-item-num">4.2.2&nbsp;&nbsp;</span>How are inventory alignment projections actually applied?</a></span></li><li><span><a href="#Apply-projection-definitions" data-toc-modified-id="Apply-projection-definitions-4.2.3"><span class="toc-item-num">4.2.3&nbsp;&nbsp;</span>Apply projection definitions</a></span></li></ul></li></ul></li><li><span><a href="#Step-2:-Generating-channel-and-(orthographic)-lexicon-distributions" data-toc-modified-id="Step-2:-Generating-channel-and-(orthographic)-lexicon-distributions-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Step 2: Generating channel and (orthographic) lexicon distributions</a></span><ul class="toc-item"><li><span><a href="#Step-2a:-Generating-channel-distributions-and-associated-metadata" data-toc-modified-id="Step-2a:-Generating-channel-distributions-and-associated-metadata-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Step 2a: Generating channel distributions and associated metadata</a></span><ul class="toc-item"><li><span><a href="#Metadata" data-toc-modified-id="Metadata-5.1.1"><span class="toc-item-num">5.1.1&nbsp;&nbsp;</span>Metadata</a></span></li><li><span><a href="#Channel-distributions" data-toc-modified-id="Channel-distributions-5.1.2"><span class="toc-item-num">5.1.2&nbsp;&nbsp;</span>Channel distributions</a></span></li></ul></li><li><span><a href="#Step-2b:-Generating-(contextual)-lexicon-distributions-over-orthographic-vocabularies" data-toc-modified-id="Step-2b:-Generating-(contextual)-lexicon-distributions-over-orthographic-vocabularies-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Step 2b: Generating (contextual) lexicon distributions over orthographic vocabularies</a></span></li></ul></li><li><span><a href="#Step-3:-Creating-combinable-models" data-toc-modified-id="Step-3:-Creating-combinable-models-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Step 3: Creating combinable models</a></span><ul class="toc-item"><li><span><a href="#Step-3a:-Filter-transcription-lexicons-to-only-include-words-that-can-be-modeled-by-a-given-channel-distribution" data-toc-modified-id="Step-3a:-Filter-transcription-lexicons-to-only-include-words-that-can-be-modeled-by-a-given-channel-distribution-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Step 3a: Filter transcription lexicons to only include words that can be modeled by a given channel distribution</a></span></li><li><span><a href="#Step-3b:-Filter-transcription-lexicons-to-only-include-words-that-are-in-a-language-model's-vocabulary" data-toc-modified-id="Step-3b:-Filter-transcription-lexicons-to-only-include-words-that-are-in-a-language-model's-vocabulary-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Step 3b: Filter transcription lexicons to only include words that are in a language model's vocabulary</a></span></li><li><span><a href="#Step-3c:-Filter-the-conditioning-events-of-channel-distributions-to-only-include-triphones-contained-in-a-transcription-lexicon's-segmental-wordform-list" data-toc-modified-id="Step-3c:-Filter-the-conditioning-events-of-channel-distributions-to-only-include-triphones-contained-in-a-transcription-lexicon's-segmental-wordform-list-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Step 3c: Filter the conditioning events of channel distributions to only include triphones contained in a transcription lexicon's segmental wordform list</a></span></li><li><span><a href="#Step-3d:-For-each-(filtered)-transcribed-lexicon-relation,-define-the-relevant-contextual-lexicon-distributions-over-orthographic-wordforms" data-toc-modified-id="Step-3d:-For-each-(filtered)-transcribed-lexicon-relation,-define-the-relevant-contextual-lexicon-distributions-over-orthographic-wordforms-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Step 3d: For each (filtered) transcribed lexicon relation, define the relevant contextual lexicon distributions over orthographic wordforms</a></span></li><li><span><a href="#Step-3e:-For-each-(filtered)-transcribed-lexicon-relation,-define-a-conditional-distribution-on-segmental-wordforms-given-an-orthographic-wordform" data-toc-modified-id="Step-3e:-For-each-(filtered)-transcribed-lexicon-relation,-define-a-conditional-distribution-on-segmental-wordforms-given-an-orthographic-wordform-6.5"><span class="toc-item-num">6.5&nbsp;&nbsp;</span>Step 3e: For each (filtered) transcribed lexicon relation, define a conditional distribution on segmental wordforms given an orthographic wordform</a></span></li></ul></li><li><span><a href="#Step-4:-Perform-forward-model-calculations" data-toc-modified-id="Step-4:-Perform-forward-model-calculations-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Step 4: Perform forward model calculations</a></span><ul class="toc-item"><li><span><a href="#Step-4a:-Define-an-incremental-channel-distribution-on-segmental-wordforms" data-toc-modified-id="Step-4a:-Define-an-incremental-channel-distribution-on-segmental-wordforms-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Step 4a: Define an incremental channel distribution on segmental wordforms</a></span></li><li><span><a href="#Step-4b:-Define-a-channel-distribution-on-(whole)-orthographic-wordforms" data-toc-modified-id="Step-4b:-Define-a-channel-distribution-on-(whole)-orthographic-wordforms-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Step 4b: Define a channel distribution on (whole) orthographic wordforms</a></span></li></ul></li><li><span><a href="#Step-5:-Generating-posterior-distributions" data-toc-modified-id="Step-5:-Generating-posterior-distributions-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Step 5: Generating posterior distributions</a></span></li><li><span><a href="#Step-6:-Generating-analysis-measures" data-toc-modified-id="Step-6:-Generating-analysis-measures-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Step 6: Generating analysis measures</a></span></li></ul></div>

# Overview

This notebook describes the processing pipeline from 
 - gating data
 - transcribed lexicon
 - a language model and (possibly empty) n-gram contexts

to 
 - channel distribution
 - lexicon distribution(s) (distributions over wordforms)
 - expected posterior distribution over intended wordform given what has been produced of what was intended.
 
 
It describes what happens at each step, checks some pre- and post-conditions, describes what you, the user must do (if anything), and scripts some commands to automatically do the necessary processing.

# Imports

In [2]:
import papermill as pm

In [3]:
from tqdm import tqdm

In [4]:
from os import getcwd, chdir, listdir, path, mkdir, makedirs

import json
import csv

In [5]:
from collections import OrderedDict

In [6]:
repo_dir = getcwd()
repo_dir

'/mnt/cube/home/AD/emeinhar/wr'

In [7]:
repo_contents_0 = listdir()
repo_contents_0

['boilerplate.py',
 'LTR_Buckeye',
 '.gitignore',
 '__pycache__',
 '1 initial directory setup.txt',
 '2a alignment_paths_and_cmds.sh',
 'Align transcriptions.ipynb',
 'LTR_CMU_destressed',
 'Gating Data - Transcription Lexicon Alignment Maker.ipynb',
 'Processing Driver Notebook.ipynb',
 'old',
 'GD_AmE',
 '.ipynb_checkpoints',
 'string_utils.py',
 'LTR_CMU_stressed',
 '.git',
 'LTR_newdic_destressed']

# Step 0: Check for foundational files

**NOTE:**
 - I assume all relevant transcriptions have been converted to Unicode IPA characters and that segment sequences have `.` as a separator. For each data source used here, the IPA alignment step is documented in a GitHub repository elsewhere. (While I don't *think* any script here depends on use of Unicode IPA symbols, I haven't - and won't - test that idea, and it is *absolutely required* that contiguous sequences of segments be separated by `.` in data files.)
 - Where language models and n-gram contexts (drawn from speech corpora) are referenced, each of these is assumed to have come from as is from other GitHub repositories.

The four steps below verify that foundational files assumed to be present by downstream notebooks are, in fact, present in the repository directory. If for some reason those files are not present, the processing pipeline will be aborted.

## Step 0a: Check for gating data

In [83]:
AmE_gating_data_dir = 'GD_AmE'
AmE_gating_data_fn = 'AmE-diphones-IPA-annotated-columns.csv'
AmE_GD_fp = path.join(AmE_gating_data_dir, AmE_gating_data_fn)
assert path.exists(AmE_gating_data_dir), 'Gating data directory {0} does not exist.'.format(AmE_gating_data_dir)
assert path.exists(AmE_GD_fp), 'Gating data file {0} does not exist.'.format(AmE_GD_fp)

The processed gating data used here come from 
 - https://github.com/emeinhardt/wmc2014-ipa
 
See those repositories for information on how they were produced.

## Step 0b: Check for transcribed lexicons

 - Each transcribed lexicon `LEXNAME` should be in a folder (e.g. `LTR_LEXNAME`) containing a file `LTR_LEXNAME.tsv`. For documentation purposes, the source file and a notebook documenting the production of the `.tsv` file should, if practicable be included in the folder as well.
   - A transcribed lexicon `LTR_....tsv` file should have two columns: `Orthographic_Wordform` and `Transcription`.
   - NB: The `LTR_` prefix on transcribed lexicon data files and containing folders is simply a convention for organization and readability, but is not required or expected by any file or script in this repository.

The assertions in the code below will only succeed if step 1 is complete for all transcribed lexicons listed for checking below.

In [49]:
newdic_destressed_ltr_folder = 'LTR_newdic_destressed'
cmu_destressed_ltr_folder = 'LTR_CMU_destressed'
cmu_stressed_ltr_folder = 'LTR_CMU_stressed'
buckeye_ltr_folder = 'LTR_Buckeye'
# nxt_swbd_ltr_folder = 

LTR_folders = (newdic_destressed_ltr_folder, cmu_destressed_ltr_folder, cmu_stressed_ltr_folder, buckeye_ltr_folder)
LTR_folders_to_process = (newdic_destressed_ltr_folder, cmu_destressed_ltr_folder, buckeye_ltr_folder)

for dirname in tqdm(LTR_folders_to_process):
    assert path.exists(dirname), 'Transcribed lexicon directory {0} not found in repo directory'.format(dirname)
    fname = path.join(dirname, dirname + '.tsv')
    assert path.exists(fname), 'Transcribed lexicon {0} not found in repo directory'.format(fname)

100%|██████████| 3/3 [00:00<00:00, 2184.15it/s]


**How are transcribed lexicon relations made?**

Each was created by processing a transcription source (a lexical database, an annotated corpus, etc.). The processing step is described in other repositories:
 - https://github.com/emeinhardt/newdic-nettalk-ipa
 - https://github.com/emeinhardt/cmu-ipa
 - https://github.com/emeinhardt/buckeye-lm
 - https://github.com/emeinhardt/switchboard-lm

Given the processed outputs of these repositories, the `Making a Transcribed Lexicon Relation - <LEXNAME>.ipynb` notebook in each `LTR_LEXNAME` folder describes how the homogeneous `.tsv` files downstream steps are created.

In [51]:
for dirname in LTR_folders_to_process:
    listdir(dirname)

['LTR_newdic_destressed.tsv',
 'Making a Transcribed Lexicon Relation - newdic_destressed.ipynb',
 'newdic_IPA.tsv',
 '.ipynb_checkpoints']

['Making a Transcribed Lexicon Relation - CMU_destressed.ipynb',
 'LTR_CMU_destressed.tsv',
 '.ipynb_checkpoints',
 'cmudict-0.7b_IPA_destressed.tsv']

['LTR_Buckeye.tsv',
 'buckeye_words_analysis_relation.json',
 '.ipynb_checkpoints',
 'Making a Transcribed Lexicon Relation - Buckeye.ipynb']

## Step 0c: Check for n-gram contexts

In [229]:
buckeye_contexts = 'buckeye_contexts.txt'
swbd2003_contexts = 'swbd2003_contexts.txt'

contexts = (buckeye_contexts, swbd2003_contexts)

for c_fn in contexts:
    assert path.exists(c_fn), "N-gram contexts file {0} does not exist.".format(c_fn)

The n-gram context files are taken from
 - https://github.com/emeinhardt/buckeye-lm
 - https://github.com/emeinhardt/switchboard-lm
 
See those repositories for more information on how the contexts were extracted. (*NB*: The context files are not included in this repository both to avoid duplication and because of licensing restrictions: to recreate these contexts, you will need access to your own copy of the Buckeye and Switchboard corpora.)

## Step 0d: Check for language model(s)

In [92]:
fisher_lm_dir = 'LM_Fisher'
fisher_lm_fn = 'fisher_utterances_main_4gram.mmap'
fisher_lm_fp = path.join(fisher_lm_dir, fisher_lm_fn)

fisher_lm_vocab_fn = 'fisher_vocabulary_main.txt'
fisher_lm_vocab_fp = path.join(fisher_lm_dir, fisher_lm_vocab_fn)

assert path.exists(fisher_lm_fp), 'Language model {0} not found'.format(fisher_lm_fp)
assert path.exists(fisher_lm_vocab_fp), 'Language model vocabulary {0} not found'.format(fisher_lm_vocab_fp)

fisher_lm_fps = {'lm':fisher_lm_fp, 
                 'vocab':fisher_lm_vocab_fp}

The (memory mapped) 4-gram language model is copied from the output of this repository:
 - https://github.com/emeinhardt/fisher-lm
 
See that repository for more information. (*NB* Again, the language model file is not included in this repository both to avoid duplication and because of licensing restrictions: to recreate these contexts, you will need access to your own copy of the Buckeye and Switchboard corpora.

# Step 1: Segment inventory alignment

## Step 1a: Define inventory alignment projections

The segment inventory of any given transcribed lexicon and the segment inventory of the gating data often do not line up. For the gating data to be usefully applied to a given lexicon of transcriptions, the strings in the (segmental) lexicon must contain only segments found in the gating data stimuli inventory.

To ensure this happens, the notebook `Gating Data - Transcription Lexicon Alignment Maker.ipynb` 
 - takes as inputs 
     - a transcribed lexicon file path and a gating data file path
     - a lexicon projection file path and a gating data projection file path
 - identifies the inventories of each and what symbols are relatively unique to the lexicon and the gating data
 - produces 
   - *a Jupyter notebook* for **you to open and finish by defining two projection functions** (i.e. Python dictionaries) to be applied to strings in the transcribed lexicon and to the gating data (one function for each). When you finish doing this (and set an export flag in the notebook to True and run the remainder of the notebook), this notebook will produce
     - two *.json files storing these projections* according to the previously provided output file paths.

The cell below will clear all existing alignment folders created using the code in this subsection:

In [201]:
%rm -rf *_aligned_w_*
%rm -rf *" alignment definition"*

The cell below will only succeed if the American English gating data of Warner, McQueen, and Cutler (2014) is contained in the repo directory with a particular directory and filename.

In [202]:
gating_data_folder = 'GD_AmE'
gating_data_fn = 'AmE-diphones-IPA-annotated-columns.csv'
gating_data_fp = path.join(gating_data_folder, gating_data_fn)

assert path.exists(gating_data_folder), 'AmE gating data folder {0} not found in repo directory'.format(gating_data_folder)
assert path.exists(gating_data_fp), 'AmE gating data {0} not found in repo directory'.format(gating_data_fp)

The third cell below will create a notebook for alignment projection definitions for each of the transcribed lexicons from the previous step and the AmE gating data.

In [11]:
def removeExtension(fp):
    dir_name = path.dirname(fp)
    file_name = path.basename(fp)
    ext = file_name.split('.')[-1]
    rest = '.'.join( file_name.split('.')[:-1] )
    return path.join(dir_name, rest)

In [203]:
arg_bundles = []
for LTR_dirname in tqdm(LTR_folders_to_process):
    LTR_fn = LTR_dirname + '.tsv'
    LTR_fp = path.join(LTR_dirname, LTR_fn)
    
    nb_output_name = 'GD_AmE-diphones - ' + LTR_dirname + ' alignment definition' + '.ipynb'
    my_g = gating_data_fp
    my_l = LTR_fp
    my_s = 'destressed'
    
    gd_alignment_dn = 'GD_AmE_' + my_s + '_' + 'aligned_w_' + LTR_dirname
    gd_alignment_fn = 'alignment_of_' + removeExtension(gating_data_fn) + '_w_' + LTR_dirname + '.json'
    gd_alignment_fp = path.join(gd_alignment_dn, gd_alignment_fn)
    if not path.exists(gd_alignment_dn):
        makedirs(gd_alignment_dn)
    my_gp = gd_alignment_fp
    
    ltr_alignment_dn = LTR_dirname + '_aligned_w_' + 'GD_AmE_' + my_s
    ltr_alignment_fn = 'alignment_of_' + LTR_dirname + '_w_' + removeExtension(gating_data_fn) + '.json'
    ltr_alignment_fp = path.join(ltr_alignment_dn, ltr_alignment_fn)
    if not path.exists(ltr_alignment_dn):
        makedirs(ltr_alignment_dn)
    my_lp = ltr_alignment_fp
    
    
    my_arg_bundle = OrderedDict({
        'LTR_dirname':LTR_dirname,
        'LTR_fn':LTR_fn,
        'LTR_fp':LTR_fp,
        'gd_alignment_dn':gd_alignment_dn,
        'gd_alignment_fn':gd_alignment_fn,
        'gd_alignment_fp':gd_alignment_fp,
        'ltr_alignment_dn':ltr_alignment_dn,
        'ltr_alignment_fn':ltr_alignment_fn,
        'ltr_alignment_fp':ltr_alignment_fp,
        'align_def_nb_output_name':nb_output_name,
        'my_g':my_g,
        'my_l':my_l,
        'my_s':my_s,
        'my_gp':my_gp,
        'my_lp':my_lp,
    })
    arg_bundles.append(my_arg_bundle)









  0%|          | 0/3 [00:00<?, ?it/s]







100%|██████████| 3/3 [00:00<00:00, 930.48it/s]

In [205]:
for arg_bundle in tqdm(arg_bundles):
    nb = pm.execute_notebook(
        'Gating Data - Transcription Lexicon Alignment Maker.ipynb',
        arg_bundle['align_def_nb_output_name'],
        parameters=dict(g = arg_bundle['my_g'], 
                        l = arg_bundle['my_l'], 
                        s = arg_bundle['my_s'], 
                        gp = arg_bundle['my_gp'], 
                        lp = arg_bundle['my_lp'])
    )
#     pm.execute_notebook(
#        'Gating Data - Transcription Lexicon Alignment Maker.ipynb',
#        nb_output_name,
#        parameters=dict(g = my_g, l = my_l, s = my_s, gp = my_gp, lp = my_lp)
#     )
    print("Finished creating alignment definition notebook '{0}'.\nOpen and run the notebook, complete the projection definition, and run the remainder of the notebook (remembering to change the export flag to 'True').\n".format(arg_bundle['align_def_nb_output_name']))









  0%|          | 0/3 [00:00<?, ?it/s]Input Notebook:  Gating Data - Transcription Lexicon Alignment Maker.ipynb
Output Notebook: GD_AmE-diphones - LTR_newdic_destressed alignment definition.ipynb









  0%|          | 0/64 [00:00<?, ?it/s]








  6%|▋         | 4/64 [00:00<00:01, 34.46it/s]








 14%|█▍        | 9/64 [00:00<00:01, 36.22it/s]








 20%|██        | 13/64 [00:00<00:01, 36.89it/s]








 27%|██▋       | 17/64 [00:00<00:01, 35.85it/s]








 33%|███▎      | 21/64 [00:00<00:01, 34.73it/s]








 39%|███▉      | 25/64 [00:00<00:01, 24.76it/s]








 44%|████▍     | 28/64 [00:00<00:01, 25.72it/s]








 48%|████▊     | 31/64 [00:01<00:01, 26.51it/s]








 53%|█████▎    | 34/64 [00:01<00:01, 22.40it/s]








 58%|█████▊    | 37/64 [00:04<00:09,  3.00it/s]








 62%|██████▎   | 40/64 [00:05<00:08,  2.81it/s]








 66%|██████▌   | 42/64 [00:05<00:05,  3.77it/s]








 70%|███████   | 45/64 [00:05<00:03,  5.09it/s]








 75%|███████▌ 

Finished creating alignment definition notebook GD_AmE-diphones - LTR_newdic_destressed alignment definition.ipynb.
Open and run the notebook, complete the projection definition, and run the remainder of the notebook (remembering to change the export flag to 'True').



Input Notebook:  Gating Data - Transcription Lexicon Alignment Maker.ipynb
Output Notebook: GD_AmE-diphones - LTR_CMU_destressed alignment definition.ipynb









  0%|          | 0/64 [00:00<?, ?it/s]








  6%|▋         | 4/64 [00:00<00:01, 31.70it/s]








 12%|█▎        | 8/64 [00:00<00:01, 33.47it/s]








 19%|█▉        | 12/64 [00:00<00:01, 34.42it/s]








 23%|██▎       | 15/64 [00:00<00:01, 31.73it/s]








 28%|██▊       | 18/64 [00:00<00:01, 30.35it/s]








 34%|███▍      | 22/64 [00:00<00:01, 30.61it/s]








 39%|███▉      | 25/64 [00:00<00:02, 19.37it/s]








 44%|████▍     | 28/64 [00:01<00:01, 21.50it/s]








 48%|████▊     | 31/64 [00:01<00:01, 22.39it/s]








 53%|█████▎    | 34/64 [00:01<00:01, 19.12it/s]








 58%|█████▊    | 37/64 [00:05<00:10,  2.50it/s]








 62%|██████▎   | 40/64 [00:06<00:09,  2.63it/s]








 66%|██████▌   | 42/64 [00:06<00:07,  2.81it/s]








 70%|███████   | 45/64 [00:06<00:04,  3.85it/s]








 75%|█

Finished creating alignment definition notebook GD_AmE-diphones - LTR_CMU_destressed alignment definition.ipynb.
Open and run the notebook, complete the projection definition, and run the remainder of the notebook (remembering to change the export flag to 'True').



Input Notebook:  Gating Data - Transcription Lexicon Alignment Maker.ipynb
Output Notebook: GD_AmE-diphones - LTR_Buckeye alignment definition.ipynb









  0%|          | 0/64 [00:00<?, ?it/s]








  6%|▋         | 4/64 [00:00<00:01, 33.95it/s]








 14%|█▍        | 9/64 [00:00<00:01, 35.78it/s]








 20%|██        | 13/64 [00:00<00:01, 36.51it/s]








 25%|██▌       | 16/64 [00:00<00:01, 33.97it/s]








 30%|██▉       | 19/64 [00:00<00:01, 32.07it/s]








 36%|███▌      | 23/64 [00:00<00:01, 32.35it/s]








 41%|████      | 26/64 [00:00<00:01, 21.99it/s]








 47%|████▋     | 30/64 [00:01<00:01, 24.26it/s]








 52%|█████▏    | 33/64 [00:01<00:01, 24.93it/s]








 56%|█████▋    | 36/64 [00:04<00:09,  2.90it/s]








 61%|██████    | 39/64 [00:04<00:06,  3.95it/s]








 66%|██████▌   | 42/64 [00:05<00:06,  3.61it/s]








 70%|███████   | 45/64 [00:05<00:03,  4.88it/s]








 75%|███████▌  | 48/64 [00:05<00:02,  6.48it/s]








 80%|███████▉

Finished creating alignment definition notebook GD_AmE-diphones - LTR_Buckeye alignment definition.ipynb.
Open and run the notebook, complete the projection definition, and run the remainder of the notebook (remembering to change the export flag to 'True').



## Step 1b: Apply inventory alignment projections

The cell below will clear all existing alignment folders created using the code in this subsection:

In [204]:
%rm -rf *" alignment application "*

### Check for projection definitions

The cell below will succeed if you have run each of the previously produced notebooks correctly and produced a projection mapping file.

In [207]:
for arg_bundle in tqdm(arg_bundles):
    args = arg_bundle
    assert path.exists(args['gd_alignment_fp']), 'Gating data alignment projection mapping not found:\n\t{0}'.format(args['gd_alignment_fp'])
    assert path.exists(args['ltr_alignment_fp']), 'Transcribed lexicon data alignment projection mapping not found:\n\t{0}'.format(args['ltr_alignment_fp'])









  0%|          | 0/3 [00:00<?, ?it/s]







100%|██████████| 3/3 [00:00<00:00, 876.49it/s]

### How are inventory alignment projections actually applied?

See `Align transcriptions.ipynb`.

### Apply projection definitions

The cell below applies each pair of alignment projections to each matched pair of gating data and transcribed lexicon choice:

In [208]:
for arg_bundle in arg_bundles:
    args = arg_bundle
    LTR_fn = args['LTR_fn']
    
    my_pg = args['my_gp']
    my_g = args['my_g']
    my_o_fn = 'GD_AmE-diphones' + '_aligned_w_' + removeExtension(LTR_fn) + '.tsv'
    my_og = path.join(args['gd_alignment_dn'], my_o_fn)
    args['align_apply_gd_nb_output_name'] = 'GD_AmE-diphones - ' + removeExtension(LTR_fn) + ' alignment application to ' + 'AmE-diphones' + '.ipynb'
    args['my_og'] = my_og
    print("Creating notebook '{0}' w/ args p, g, o = \n\t{1}\n\t{2}\n\t{3}".format(args['align_apply_gd_nb_output_name'], my_pg, my_g, my_og))
    nb = pm.execute_notebook(
        'Align transcriptions.ipynb',
        args['align_apply_gd_nb_output_name'],
        parameters=dict(p = my_pg,
                        g = my_g,
                        o = my_og)
    )
    print('Finished applying alignment projection\n\tp = {0}\nto\n\tg = {1}\nResult saved to\n\t{2}'.format(my_pg, my_g, my_og))
    print(' ')
    
    my_pl = args['my_lp']
    my_l = args['my_l']
    my_o_fn = removeExtension(LTR_fn) + '_aligned_w_' + 'GD_AmE-diphones' + '.tsv'
    my_ol = path.join(args['ltr_alignment_dn'], my_o_fn)
    args['align_apply_ltr_nb_output_name'] = 'GD_AmE-diphones - ' + removeExtension(LTR_fn) + ' alignment application to ' + removeExtension(LTR_fn) + '.ipynb'
    print('Creating notebook {0} w/ args p, g, o = \n\t{1}\n\t{2}\n\t{3}'.format(args['align_apply_ltr_nb_output_name'], my_pg, my_l, my_ol))
    args['my_ol'] = my_ol
    nb = pm.execute_notebook(
        'Align transcriptions.ipynb',
        args['align_apply_ltr_nb_output_name'],
        parameters=dict(p = my_pl,
                        l = my_l,
                        o = my_ol)
    )
    print('Finished applying alignment projection\n\tp = {0}\nto\n\tl = {1}\nResult saved to\n\t{2}'.format(my_pl, my_l, my_ol))
    print('\n')

Creating notebook 'GD_AmE-diphones - LTR_newdic_destressed alignment application to AmE-diphones.ipynb' w/ args p, g, o = 
	GD_AmE_destressed_aligned_w_LTR_newdic_destressed/alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_newdic_destressed.json
	GD_AmE/AmE-diphones-IPA-annotated-columns.csv
	GD_AmE_destressed_aligned_w_LTR_newdic_destressed/GD_AmE-diphones_aligned_w_LTR_newdic_destressed.tsv


Input Notebook:  Align transcriptions.ipynb
Output Notebook: GD_AmE-diphones - LTR_newdic_destressed alignment application to AmE-diphones.ipynb








  0%|          | 0/64 [00:00<?, ?it/s]







  6%|▋         | 4/64 [00:00<00:01, 38.06it/s]







 14%|█▍        | 9/64 [00:00<00:01, 38.87it/s]







 19%|█▉        | 12/64 [00:00<00:02, 25.69it/s]







 23%|██▎       | 15/64 [00:00<00:02, 21.30it/s]







 30%|██▉       | 19/64 [00:00<00:01, 24.08it/s]







 36%|███▌      | 23/64 [00:00<00:01, 26.47it/s]







 42%|████▏     | 27/64 [00:00<00:01, 28.96it/s]







 48%|████▊     | 31/64 [00:01<00:01, 30.10it/s]







 55%|█████▍    | 35/64 [00:04<00:07,  3.95it/s]







 61%|██████    | 39/64 [00:04<00:04,  5.08it/s]







 66%|██████▌   | 42/64 [00:04<00:03,  6.76it/s]







 70%|███████   | 45/64 [00:04<00:02,  8.76it/s]







 77%|███████▋  | 49/64 [00:04<00:01, 11.20it/s]







 83%|████████▎ | 53/64 [00:04<00:00, 13.84it/s]







 89%|████████▉ | 57/64 [00:05<00:

Finished applying alignment projection
	p = GD_AmE_destressed_aligned_w_LTR_newdic_destressed/alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_newdic_destressed.json
to
	g = GD_AmE/AmE-diphones-IPA-annotated-columns.csv
Result saved to
	GD_AmE_destressed_aligned_w_LTR_newdic_destressed/GD_AmE-diphones_aligned_w_LTR_newdic_destressed.tsv
 
Creating notebook GD_AmE-diphones - LTR_newdic_destressed alignment application to LTR_newdic_destressed.ipynb w/ args p, g, o = 
	GD_AmE_destressed_aligned_w_LTR_newdic_destressed/alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_newdic_destressed.json
	LTR_newdic_destressed/LTR_newdic_destressed.tsv
	LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_w_GD_AmE-diphones.tsv


Input Notebook:  Align transcriptions.ipynb
Output Notebook: GD_AmE-diphones - LTR_newdic_destressed alignment application to LTR_newdic_destressed.ipynb








  0%|          | 0/64 [00:00<?, ?it/s]







  6%|▋         | 4/64 [00:00<00:01, 37.69it/s]







 14%|█▍        | 9/64 [00:00<00:01, 38.94it/s]







 19%|█▉        | 12/64 [00:00<00:02, 25.70it/s]







 23%|██▎       | 15/64 [00:00<00:02, 21.41it/s]







 30%|██▉       | 19/64 [00:00<00:01, 23.97it/s]







 36%|███▌      | 23/64 [00:00<00:01, 26.11it/s]







 42%|████▏     | 27/64 [00:00<00:01, 28.03it/s]







 48%|████▊     | 31/64 [00:01<00:01, 28.75it/s]







 53%|█████▎    | 34/64 [00:01<00:01, 28.20it/s]







 58%|█████▊    | 37/64 [00:01<00:01, 24.43it/s]







 64%|██████▍   | 41/64 [00:01<00:00, 26.75it/s]







 70%|███████   | 45/64 [00:01<00:00, 28.62it/s]







 75%|███████▌  | 48/64 [00:01<00:00, 28.39it/s]







 81%|████████▏ | 52/64 [00:02<00:00, 20.68it/s]







 86%|████████▌ | 55/64 [

Finished applying alignment projection
	p = LTR_newdic_destressed_aligned_w_GD_AmE_destressed/alignment_of_LTR_newdic_destressed_w_AmE-diphones-IPA-annotated-columns.json
to
	l = LTR_newdic_destressed/LTR_newdic_destressed.tsv
Result saved to
	LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_w_GD_AmE-diphones.tsv


Creating notebook 'GD_AmE-diphones - LTR_CMU_destressed alignment application to AmE-diphones.ipynb' w/ args p, g, o = 
	GD_AmE_destressed_aligned_w_LTR_CMU_destressed/alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_CMU_destressed.json
	GD_AmE/AmE-diphones-IPA-annotated-columns.csv
	GD_AmE_destressed_aligned_w_LTR_CMU_destressed/GD_AmE-diphones_aligned_w_LTR_CMU_destressed.tsv


...

Finished applying alignment projection
	p = LTR_CMU_destressed_aligned_w_GD_AmE_destressed/alignment_of_LTR_CMU_destressed_w_AmE-diphones-IPA-annotated-columns.json
to
	l = LTR_CMU_destressed/LTR_CMU_destressed.tsv
Result saved to
	LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_w_GD_AmE-diphones.tsv


Creating notebook 'GD_AmE-diphones - LTR_Buckeye alignment application to AmE-diphones.ipynb' w/ args p, g, o = 
	GD_AmE_destressed_aligned_w_LTR_Buckeye/alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_Buckeye.json
	GD_AmE/AmE-diphones-IPA-annotated-columns.csv
	GD_AmE_destressed_aligned_w_LTR_Buckeye/GD_AmE-diphones_aligned_w_LTR_Buckeye.tsv


Input Notebook:  Align transcriptions.ipynb
Output Notebook: GD_AmE-diphones - LTR_Buckeye alignment application to AmE-diphones.ipynb








  0%|          | 0/64 [00:00<?, ?it/s]







  6%|▋         | 4/64 [00:00<00:01, 36.91it/s]







 14%|█▍        | 9/64 [00:00<00:01, 38.07it/s]







 19%|█▉        | 12/64 [00:00<00:02, 24.97it/s]







 23%|██▎       | 15/64 [00:00<00:02, 21.04it/s]







 30%|██▉       | 19/64 [00:00<00:01, 24.09it/s]







 36%|███▌      | 23/64 [00:00<00:01, 26.66it/s]







 42%|████▏     | 27/64 [00:00<00:01, 28.82it/s]







 48%|████▊     | 31/64 [00:01<00:01, 30.31it/s]







 55%|█████▍    | 35/64 [00:04<00:07,  3.93it/s]







 61%|██████    | 39/64 [00:04<00:04,  5.35it/s]







 67%|██████▋   | 43/64 [00:04<00:02,  7.13it/s]







 72%|███████▏  | 46/64 [00:04<00:01,  9.22it/s]







 78%|███████▊  | 50/64 [00:04<00:01, 11.73it/s]







 84%|████████▍ | 54/64 [00:04<00:00, 14.37it/s]







 91%|█████████ | 58/64 [00:04<00:00, 16.46i

Finished applying alignment projection
	p = GD_AmE_destressed_aligned_w_LTR_Buckeye/alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_Buckeye.json
to
	g = GD_AmE/AmE-diphones-IPA-annotated-columns.csv
Result saved to
	GD_AmE_destressed_aligned_w_LTR_Buckeye/GD_AmE-diphones_aligned_w_LTR_Buckeye.tsv
 
Creating notebook GD_AmE-diphones - LTR_Buckeye alignment application to LTR_Buckeye.ipynb w/ args p, g, o = 
	GD_AmE_destressed_aligned_w_LTR_Buckeye/alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_Buckeye.json
	LTR_Buckeye/LTR_Buckeye.tsv
	LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_w_GD_AmE-diphones.tsv


Input Notebook:  Align transcriptions.ipynb
Output Notebook: GD_AmE-diphones - LTR_Buckeye alignment application to LTR_Buckeye.ipynb








  0%|          | 0/64 [00:00<?, ?it/s]







  8%|▊         | 5/64 [00:00<00:01, 41.65it/s]







 16%|█▌        | 10/64 [00:00<00:01, 41.67it/s]







 20%|██        | 13/64 [00:00<00:02, 25.08it/s]







 23%|██▎       | 15/64 [00:00<00:02, 19.28it/s]







 30%|██▉       | 19/64 [00:00<00:02, 21.85it/s]







 36%|███▌      | 23/64 [00:00<00:01, 24.07it/s]







 42%|████▏     | 27/64 [00:01<00:01, 26.35it/s]







 48%|████▊     | 31/64 [00:01<00:01, 28.12it/s]







 53%|█████▎    | 34/64 [00:01<00:01, 25.92it/s]







 58%|█████▊    | 37/64 [00:01<00:01, 25.96it/s]







 64%|██████▍   | 41/64 [00:01<00:00, 27.50it/s]







 70%|███████   | 45/64 [00:01<00:00, 28.63it/s]







 75%|███████▌  | 48/64 [00:01<00:00, 28.67it/s]







 80%|███████▉  | 51/64 [00:01<00:00, 28.66it/s]







 84%|████████▍ | 54/64 [00:01<00:00, 26.18i

Finished applying alignment projection
	p = LTR_Buckeye_aligned_w_GD_AmE_destressed/alignment_of_LTR_Buckeye_w_AmE-diphones-IPA-annotated-columns.json
to
	l = LTR_Buckeye/LTR_Buckeye.tsv
Result saved to
	LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_w_GD_AmE-diphones.tsv




# Step 2: Generating channel and (orthographic) lexicon distributions

#FIXME

I. **Channel distribution**

 A. Given gating data, we can define uniphone and triphone channel distributions.
 
II. **Lexicon distribution**

A. **Alignment of the vocabulary with the channel model and with a language model, alignment of the channel model with a lexicon**
 1. Given a transcribed lexicon relation and information about the conditioning events and sample space of a channel model, we can project a transcribed lexicon relation down to just what can be modeled using the channel model.
 2. Given the vocabulary of an (orthographic) $n$-gram model, we can project a transcribed lexicon relation down to just what can be aligned with the vocabulary of the $n$-gram model. (I.e. if the $n$-gram model vocabulary is a unary relation on orthographic wordforms, this calculates the *natural join* of the $n$-gram model vocabulary and the transcribed lexicon relation.)
 3. Given a channel model and a transcribed lexicon relation that the channel model will be applied to, we can remove $n$-phones from the channel model that do not appear in the transcribed lexicon relation.

B. **Distribution construction**
 1. Given a transcribed lexicon relation, we can define a family of conditional distributions on segmental wordforms given a choice of orthographic wordform: e.g. if orthographic wordform $w$ can surface as all and only the segmental sequences $s_0$ and $s_1$, then we construct a uniform distribution over $s_0$ and $s_1$ given $w$.
 2. Given a (unigram) distribution over orthographic wordforms and orthographic->segmental wordform conditional distributions, we can define a distribution over segmental wordforms.
 3. Given an $n$-gram model, the vocabulary of the model, a set of (orthographic) $n$-gram contexts $C$, and a target vocabulary of orthographic wordforms of interest (i.e. the ones we have transcriptions for), we can calculate a distribution over the target vocabulary for each $n$-gram context.

In [40]:
arg_bundle

OrderedDict([('LTR_dirname', 'LTR_Buckeye'),
             ('LTR_fn', 'LTR_Buckeye.tsv'),
             ('LTR_fp', 'LTR_Buckeye/LTR_Buckeye.tsv'),
             ('gd_alignment_dn', 'GD_AmE_destressed_aligned_w_LTR_Buckeye'),
             ('gd_alignment_fn',
              'alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_Buckeye.json'),
             ('gd_alignment_fp',
              'GD_AmE_destressed_aligned_w_LTR_Buckeye/alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_Buckeye.json'),
             ('ltr_alignment_dn', 'LTR_Buckeye_aligned_w_GD_AmE_destressed'),
             ('ltr_alignment_fn',
              'alignment_of_LTR_Buckeye_w_AmE-diphones-IPA-annotated-columns.json'),
             ('ltr_alignment_fp',
              'LTR_Buckeye_aligned_w_GD_AmE_destressed/alignment_of_LTR_Buckeye_w_AmE-diphones-IPA-annotated-columns.json'),
             ('align_def_nb_output_name',
              'GD_AmE-diphones - LTR_Buckeye alignment definition.ipynb'),
             ('my_g', 'GD_AmE

## Step 2a: Generating channel distributions and associated metadata

In [23]:
%ls -d GD_*

 GD_AmE/
 GD_AmE_destressed_aligned_w_LTR_Buckeye/
 GD_AmE_destressed_aligned_w_LTR_CMU_destressed/
 GD_AmE_destressed_aligned_w_LTR_newdic_destressed/
'GD_AmE-diphones - LTR_Buckeye alignment application to AmE-diphones.ipynb'
'GD_AmE-diphones - LTR_Buckeye alignment application to LTR_Buckeye.ipynb'
'GD_AmE-diphones - LTR_Buckeye alignment definition.ipynb'
'GD_AmE-diphones - LTR_CMU_destressed alignment application to AmE-diphones.ipynb'
'GD_AmE-diphones - LTR_CMU_destressed alignment application to LTR_CMU_destressed.ipynb'
'GD_AmE-diphones - LTR_CMU_destressed alignment definition.ipynb'
'GD_AmE-diphones - LTR_newdic_destressed alignment application to AmE-diphones.ipynb'
'GD_AmE-diphones - LTR_newdic_destressed alignment application to LTR_newdic_destressed.ipynb'
'GD_AmE-diphones - LTR_newdic_destressed alignment definition.ipynb'


In [27]:
gating_data_folders = ('GD_AmE', ) + tuple(map(lambda ab: ab['gd_alignment_dn'], arg_bundles))
gating_data_folders

('GD_AmE',
 'GD_AmE_destressed_aligned_w_LTR_newdic_destressed',
 'GD_AmE_destressed_aligned_w_LTR_CMU_destressed',
 'GD_AmE_destressed_aligned_w_LTR_Buckeye')

In [44]:
gating_data_fps = ('GD_AmE/AmE-diphones-IPA-annotated-columns.csv',) + \
                  tuple(map(lambda ab: ab['my_og'], arg_bundles))
gating_data_fps

('GD_AmE/AmE-diphones-IPA-annotated-columns.csv',
 'GD_AmE_destressed_aligned_w_LTR_newdic_destressed/GD_AmE-diphones_aligned_w_LTR_newdic_destressed.tsv',
 'GD_AmE_destressed_aligned_w_LTR_CMU_destressed/GD_AmE-diphones_aligned_w_LTR_CMU_destressed.tsv',
 'GD_AmE_destressed_aligned_w_LTR_Buckeye/GD_AmE-diphones_aligned_w_LTR_Buckeye.tsv')

### Metadata

First (for downstream convenience) we identify the $n$-phones (not) contained in and (not) constructible from each of the versions of the gating data.

**NB:** This is done by calling the notebook `Run n-phone analysis of gating data.ipynb`, passing it **the filepath to a gating data `.csv`/`.tsv` file** and **a path to an output directory** for the dozen or so files the notebook will produce.

In [217]:
for gating_data_fp in gating_data_fps:
    gd_dir = path.dirname(gating_data_fp)
    nb = pm.execute_notebook(
        'Run n-phone analysis of gating data.ipynb',
#         args['align_apply_ltr_nb_output_name'],
        path.join(gd_dir, gd_dir) + " n-phone analysis.ipynb",
        parameters=dict(g = gating_data_fp,
                        o = gd_dir)
    )
    listdir(gd_dir)
    print('\n')

Input Notebook:  Run n-phone analysis of gating data.ipynb
Output Notebook: GD_AmE/GD_AmE n-phone analysis.ipynb









  0%|          | 0/43 [00:00<?, ?it/s]








 14%|█▍        | 6/43 [00:00<00:00, 56.41it/s]








 28%|██▊       | 12/43 [00:00<00:00, 57.33it/s]








 37%|███▋      | 16/43 [00:00<00:01, 22.52it/s]








 49%|████▉     | 21/43 [00:00<00:00, 26.65it/s]








 60%|██████    | 26/43 [00:00<00:00, 29.25it/s]








 70%|██████▉   | 30/43 [00:03<00:03,  4.14it/s]








 77%|███████▋  | 33/43 [00:04<00:02,  4.10it/s]








 84%|████████▎ | 36/43 [00:06<00:02,  2.56it/s]








 84%|████████▎ | 36/43 [00:26<00:02,  2.56it/s]








 93%|█████████▎| 40/43 [00:26<00:05,  1.78s/it]








 98%|█████████▊| 42/43 [00:27<00:01,  1.28s/it]








100%|██████████| 43/43 [00:27<00:00,  1.59it/s]

['destressed response diphones.txt',
 'destressed stimuli diphone-based constructible triphones.txt',
 'stressed stimuli diphone-based constructible triphones.txt',
 'stressed stimuli uniphones.txt',
 'destressed stimuli uniphones.txt',
 'destressed stimuli illegal diphones.txt',
 'AmE-diphones-IPA-annotated-columns.csv',
 'destressed response illegal diphones.txt',
 'destressed response diphone-based constructible triphones.txt',
 'stressed stimuli illegal diphones.txt',
 'destressed stimuli diphone-based illegal triphones.txt',
 'destressed stimuli diphones.txt',
 'destressed response uniphones.txt',
 'stressed stimuli diphones.txt',
 'stressed stimuli diphone-based illegal triphones.txt',
 '.ipynb_checkpoints',
 'destressed response diphone-based illegal triphones.txt',
 'GD_AmE n-phone analysis.ipynb']

Input Notebook:  Run n-phone analysis of gating data.ipynb
Output Notebook: GD_AmE_destressed_aligned_w_LTR_newdic_destressed/GD_AmE_destressed_aligned_w_LTR_newdic_destressed n-phone analysis.ipynb









  0%|          | 0/43 [00:00<?, ?it/s]








 12%|█▏        | 5/43 [00:00<00:00, 43.61it/s]








 26%|██▌       | 11/43 [00:00<00:00, 47.23it/s]








 37%|███▋      | 16/43 [00:00<00:00, 35.25it/s]








 49%|████▉     | 21/43 [00:00<00:00, 36.51it/s]








 60%|██████    | 26/43 [00:00<00:00, 35.89it/s]








 70%|██████▉   | 30/43 [00:03<00:03,  4.19it/s]








 77%|███████▋  | 33/43 [00:04<00:02,  4.11it/s]








 84%|████████▎ | 36/43 [00:06<00:02,  2.52it/s]








 84%|████████▎ | 36/43 [00:26<00:02,  2.52it/s]








 93%|█████████▎| 40/43 [00:26<00:05,  1.80s/it]








 98%|█████████▊| 42/43 [00:27<00:01,  1.29s/it]








100%|██████████| 43/43 [00:27<00:00,  1.58it/s]

['destressed response diphones.txt',
 'destressed stimuli diphone-based illegal triphones.txt',
 'destressed stimuli uniphones.txt',
 'stressed stimuli diphones.txt',
 'GD_AmE_destressed_aligned_w_LTR_newdic_destressed n-phone analysis.ipynb',
 'alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_newdic_destressed.json',
 'stressed stimuli illegal diphones.txt',
 'stressed stimuli diphone-based constructible triphones.txt',
 'destressed response illegal diphones.txt',
 'destressed response uniphones.txt',
 'destressed stimuli illegal diphones.txt',
 'destressed response diphone-based illegal triphones.txt',
 'destressed response diphone-based constructible triphones.txt',
 'GD_AmE-diphones_aligned_w_LTR_newdic_destressed.tsv',
 'stressed stimuli diphone-based illegal triphones.txt',
 'stressed stimuli uniphones.txt',
 'destressed stimuli diphone-based constructible triphones.txt',
 'destressed stimuli diphones.txt']

Input Notebook:  Run n-phone analysis of gating data.ipynb
Output Notebook: GD_AmE_destressed_aligned_w_LTR_CMU_destressed/GD_AmE_destressed_aligned_w_LTR_CMU_destressed n-phone analysis.ipynb









  0%|          | 0/43 [00:00<?, ?it/s]








 14%|█▍        | 6/43 [00:00<00:00, 54.19it/s]








 28%|██▊       | 12/43 [00:00<00:00, 55.04it/s]








 37%|███▋      | 16/43 [00:00<00:00, 33.34it/s]








 49%|████▉     | 21/43 [00:00<00:00, 36.57it/s]








 60%|██████    | 26/43 [00:00<00:00, 36.42it/s]








 70%|██████▉   | 30/43 [00:03<00:03,  4.09it/s]








 77%|███████▋  | 33/43 [00:04<00:02,  4.01it/s]








 84%|████████▎ | 36/43 [00:06<00:02,  2.51it/s]








 84%|████████▎ | 36/43 [00:17<00:02,  2.51it/s]








 93%|█████████▎| 40/43 [00:26<00:05,  1.80s/it]








 98%|█████████▊| 42/43 [00:27<00:01,  1.29s/it]








100%|██████████| 43/43 [00:27<00:00,  1.58it/s]

['destressed stimuli uniphones.txt',
 'destressed response diphones.txt',
 'alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_CMU_destressed.json',
 'destressed stimuli diphones.txt',
 'destressed response diphone-based illegal triphones.txt',
 'destressed response illegal diphones.txt',
 'stressed stimuli diphone-based illegal triphones.txt',
 'destressed response uniphones.txt',
 'stressed stimuli uniphones.txt',
 'GD_AmE_destressed_aligned_w_LTR_CMU_destressed n-phone analysis.ipynb',
 'stressed stimuli diphone-based constructible triphones.txt',
 'stressed stimuli illegal diphones.txt',
 'destressed stimuli diphone-based constructible triphones.txt',
 'destressed response diphone-based constructible triphones.txt',
 'destressed stimuli illegal diphones.txt',
 'destressed stimuli diphone-based illegal triphones.txt',
 'stressed stimuli diphones.txt',
 'GD_AmE-diphones_aligned_w_LTR_CMU_destressed.tsv']

Input Notebook:  Run n-phone analysis of gating data.ipynb
Output Notebook: GD_AmE_destressed_aligned_w_LTR_Buckeye/GD_AmE_destressed_aligned_w_LTR_Buckeye n-phone analysis.ipynb









  0%|          | 0/43 [00:00<?, ?it/s]








 14%|█▍        | 6/43 [00:00<00:00, 56.81it/s]








 28%|██▊       | 12/43 [00:00<00:00, 56.19it/s]








 37%|███▋      | 16/43 [00:00<00:00, 32.30it/s]








 49%|████▉     | 21/43 [00:00<00:00, 35.72it/s]








 58%|█████▊    | 25/43 [00:00<00:00, 36.73it/s]








 70%|██████▉   | 30/43 [00:03<00:02,  5.11it/s]








 77%|███████▋  | 33/43 [00:04<00:02,  4.66it/s]








 84%|████████▎ | 36/43 [00:06<00:02,  2.72it/s]








 84%|████████▎ | 36/43 [00:18<00:02,  2.72it/s]








 93%|█████████▎| 40/43 [00:26<00:05,  1.73s/it]








 98%|█████████▊| 42/43 [00:26<00:01,  1.24s/it]








100%|██████████| 43/43 [00:26<00:00,  1.63it/s]

['destressed response diphone-based constructible triphones.txt',
 'stressed stimuli illegal diphones.txt',
 'GD_AmE-diphones_aligned_w_LTR_Buckeye.tsv',
 'destressed response diphones.txt',
 'destressed response diphone-based illegal triphones.txt',
 'destressed stimuli uniphones.txt',
 'destressed stimuli illegal diphones.txt',
 'stressed stimuli diphone-based illegal triphones.txt',
 'destressed stimuli diphones.txt',
 'GD_AmE_destressed_aligned_w_LTR_Buckeye n-phone analysis.ipynb',
 'stressed stimuli diphones.txt',
 'destressed response uniphones.txt',
 'destressed stimuli diphone-based illegal triphones.txt',
 'destressed response illegal diphones.txt',
 'destressed stimuli diphone-based constructible triphones.txt',
 'stressed stimuli uniphones.txt',
 'alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_Buckeye.json',
 'stressed stimuli diphone-based constructible triphones.txt']

### Channel distributions

Next, the notebook `Producing channel distributions.ipynb` will create `.json` files defining (among other things) a uniphone and triphone channel distribution. It requires the following arguments to specify information about what kind of channel model to build and where to put it:
 - **a filepath** to a gating data file
 - **a directory** containing metadata indicating possible/impossible $n$-phones
 - **a string argument** ("stressed" or "destressed") indicating whether the distribution will be over a segment inventory with or without stress information
 - **a real valued**  smoothing parameter (a pseudocount to add to every channel outcome)
 - **an output directory** to write the channel model to.

In [53]:
pseudocounts = (0, 0.01, 0.05, 0.1)

In [79]:
cm_arg_bundles = []
for gating_data_fp in gating_data_fps:
    metadata_dir = path.dirname(gating_data_fp)
    s = "destressed"
    channel_model_dir_stem = 'CM' + metadata_dir[2:]
    
    for pc in pseudocounts:
        channel_model_dir_suffix = '_pseudocount' + str(pc)
        if metadata_dir == 'GD_AmE':
            channel_model_dir = channel_model_dir_stem + '_' + s + '_unaligned' + channel_model_dir_suffix
        else:
            channel_model_dir = channel_model_dir_stem + channel_model_dir_suffix
        nb_output_name = 'Producing channel distributions from ' + metadata_dir + ', pc={0}'.format(pc) + '.ipynb'
        new_arg_bundle = {'gating_data_fp':gating_data_fp,
                          'metadata_dir':metadata_dir,
                          's':s,
                          'c':pc,
                          'cm_dir':channel_model_dir,
                          'nb_output_name':nb_output_name}
        cm_arg_bundles.append(new_arg_bundle)

In [80]:
cm_arg_bundles[0]

{'gating_data_fp': 'GD_AmE/AmE-diphones-IPA-annotated-columns.csv',
 'metadata_dir': 'GD_AmE',
 's': 'destressed',
 'c': 0,
 'cm_dir': 'CM_AmE_destressed_unaligned_pseudocount0',
 'nb_output_name': 'Producing channel distributions from GD_AmE, pc=0.ipynb'}

In [218]:
#takes ~100m on wittgenstein
for ab in cm_arg_bundles:
    if not path.exists(ab['cm_dir']):
        mkdir(ab['cm_dir'])
    
    nb = pm.execute_notebook(
    'Producing channel distributions.ipynb',
    path.join(ab['cm_dir'], ab['nb_output_name']),
    parameters=dict(g = ab['gating_data_fp'],
                    m = ab['metadata_dir'],
                    s = ab['s'],
                    c = ab['c'],
                    o = ab['cm_dir'])
    )
    print('\n')

Input Notebook:  Producing channel distributions.ipynb
Output Notebook: CM_AmE_destressed_unaligned_pseudocount0/Producing channel distributions from GD_AmE, pc=0.ipynb









  0%|          | 0/189 [00:00<?, ?it/s]








  1%|          | 2/189 [00:00<00:15, 12.46it/s]








  2%|▏         | 4/189 [00:00<00:14, 12.96it/s]








  3%|▎         | 6/189 [00:00<00:13, 13.25it/s]








  4%|▍         | 8/189 [00:00<00:13, 13.33it/s]








  5%|▌         | 10/189 [00:00<00:13, 13.42it/s]








  6%|▋         | 12/189 [00:00<00:12, 13.69it/s]








  7%|▋         | 14/189 [00:01<00:12, 13.79it/s]








  8%|▊         | 16/189 [00:01<00:12, 13.41it/s]








 10%|▉         | 18/189 [00:01<00:12, 13.49it/s]








 11%|█         | 20/189 [00:01<00:17,  9.72it/s]








 12%|█▏        | 22/189 [00:01<00:15, 10.64it/s]








 13%|█▎        | 24/189 [00:01<00:14, 11.33it/s]








 14%|█▍        | 26/189 [00:02<00:13, 11.95it/s]








 15%|█▍        | 28/189 [00:02<00:13,

Input Notebook:  Producing channel distributions.ipynb
Output Notebook: CM_AmE_destressed_unaligned_pseudocount0.01/Producing channel distributions from GD_AmE, pc=0.01.ipynb









  0%|          | 0/189 [00:00<?, ?it/s]








  1%|          | 2/189 [00:00<00:14, 12.79it/s]








  2%|▏         | 4/189 [00:00<00:13, 13.23it/s]








  3%|▎         | 6/189 [00:00<00:13, 13.41it/s]








  4%|▍         | 8/189 [00:00<00:13, 13.72it/s]








  5%|▌         | 10/189 [00:00<00:13, 13.57it/s]








  6%|▋         | 12/189 [00:00<00:13, 13.49it/s]








  7%|▋         | 14/189 [00:01<00:13, 13.45it/s]








  8%|▊         | 16/189 [00:01<00:12, 13.47it/s]








 10%|▉         | 18/189 [00:01<00:12, 13.69it/s]








 11%|█         | 20/189 [00:01<00:16, 10.44it/s]








 12%|█▏        | 22/189 [00:01<00:14, 11.32it/s]








 13%|█▎        | 24/189 [00:01<00:13, 12.10it/s]








 14%|█▍        | 26/189 [00:02<00:12, 12.65it/s]








 15%|█▍        | 28/189 [00:02<

Input Notebook:  Producing channel distributions.ipynb
Output Notebook: CM_AmE_destressed_unaligned_pseudocount0.05/Producing channel distributions from GD_AmE, pc=0.05.ipynb









  0%|          | 0/189 [00:00<?, ?it/s]








  1%|          | 1/189 [00:00<00:21,  8.81it/s]








  2%|▏         | 3/189 [00:00<00:18,  9.91it/s]








  3%|▎         | 5/189 [00:00<00:17, 10.82it/s]








  4%|▎         | 7/189 [00:00<00:15, 11.38it/s]








  5%|▍         | 9/189 [00:00<00:14, 12.14it/s]








  6%|▌         | 11/189 [00:00<00:14, 12.66it/s]








  7%|▋         | 13/189 [00:00<00:13, 12.87it/s]








  8%|▊         | 15/189 [00:01<00:13, 12.82it/s]








  9%|▉         | 17/189 [00:01<00:13, 13.01it/s]








 10%|█         | 19/189 [00:01<00:15, 10.72it/s]








 11%|█         | 21/189 [00:01<00:14, 11.26it/s]








 12%|█▏        | 23/189 [00:01<00:14, 11.76it/s]








 13%|█▎        | 25/189 [00:02<00:13, 12.37it/s]








 14%|█▍        | 27/189 [00:02<0

 85%|████████▌ | 161/189 [03:54<00:03,  8.48it/s]








 86%|████████▌ | 162/189 [03:54<00:03,  8.76it/s]








 86%|████████▌ | 163/189 [03:54<00:02,  8.71it/s]








 87%|████████▋ | 164/189 [03:55<00:02,  8.81it/s]








 87%|████████▋ | 165/189 [03:55<00:02,  9.08it/s]








 88%|████████▊ | 166/189 [03:55<00:02,  9.17it/s]








 89%|████████▉ | 168/189 [03:55<00:02,  9.37it/s]








 90%|████████▉ | 170/189 [03:55<00:01,  9.60it/s]








 90%|█████████ | 171/189 [03:55<00:01,  9.57it/s]








 91%|█████████ | 172/189 [04:53<04:53, 17.24s/it]








 92%|█████████▏| 173/189 [04:53<03:13, 12.10s/it]








 92%|█████████▏| 174/189 [04:53<02:07,  8.50s/it]








 93%|█████████▎| 175/189 [04:53<01:23,  5.98s/it]








 93%|█████████▎| 176/189 [04:53<00:56,  4.35s/it]








 94%|█████████▍| 178/189 [04:54<00:33,  3.07s/it]








 95%|█████████▌| 180/189 [07:49<04:16, 28.47s/it]








 96%|█████████▌| 181/189 [07:51<02:45, 20.66s/it]








 96%|█████████

Input Notebook:  Producing channel distributions.ipynb
Output Notebook: CM_AmE_destressed_unaligned_pseudocount0.1/Producing channel distributions from GD_AmE, pc=0.1.ipynb









  0%|          | 0/189 [00:00<?, ?it/s]








  1%|          | 2/189 [00:00<00:15, 12.18it/s]








  2%|▏         | 4/189 [00:00<00:14, 12.77it/s]








  3%|▎         | 6/189 [00:00<00:13, 13.22it/s]








  4%|▍         | 8/189 [00:00<00:13, 13.56it/s]








  5%|▌         | 10/189 [00:00<00:12, 13.82it/s]








  6%|▋         | 12/189 [00:00<00:12, 13.98it/s]








  7%|▋         | 14/189 [00:01<00:12, 13.70it/s]








  8%|▊         | 16/189 [00:01<00:12, 13.61it/s]








 10%|▉         | 18/189 [00:01<00:12, 13.60it/s]








 11%|█         | 20/189 [00:01<00:15, 10.99it/s]








 12%|█▏        | 22/189 [00:01<00:14, 11.71it/s]








 13%|█▎        | 24/189 [00:01<00:13, 12.32it/s]








 14%|█▍        | 26/189 [00:02<00:13, 12.46it/s]








 15%|█▍        | 28/189 [00:02<00

...

Input Notebook:  Producing channel distributions.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/Producing channel distributions from GD_AmE_destressed_aligned_w_LTR_newdic_destressed, pc=0.05.ipynb









  0%|          | 0/189 [00:00<?, ?it/s]








  1%|          | 2/189 [00:00<00:13, 13.97it/s]








  2%|▏         | 4/189 [00:00<00:13, 14.11it/s]








  3%|▎         | 6/189 [00:00<00:13, 13.61it/s]








  4%|▍         | 8/189 [00:00<00:13, 13.63it/s]








  5%|▌         | 10/189 [00:00<00:13, 13.75it/s]








  6%|▋         | 12/189 [00:00<00:12, 13.91it/s]








  7%|▋         | 14/189 [00:01<00:12, 13.68it/s]








  8%|▊         | 16/189 [00:01<00:12, 13.58it/s]








 10%|▉         | 18/189 [00:01<00:12, 13.61it/s]








 11%|█         | 20/189 [00:01<00:15, 11.08it/s]








 12%|█▏        | 22/189 [00:01<00:14, 11.56it/s]








 13%|█▎        | 24/189 [00:01<00:13, 12.18it/s]








 14%|█▍        | 26/189

Input Notebook:  Producing channel distributions.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/Producing channel distributions from GD_AmE_destressed_aligned_w_LTR_newdic_destressed, pc=0.1.ipynb









  0%|          | 0/189 [00:00<?, ?it/s]








  1%|          | 2/189 [00:00<00:14, 13.10it/s]








  2%|▏         | 4/189 [00:00<00:13, 13.31it/s]








  3%|▎         | 6/189 [00:00<00:13, 13.39it/s]








  4%|▍         | 8/189 [00:00<00:13, 13.50it/s]








  5%|▌         | 10/189 [00:00<00:13, 13.58it/s]








  6%|▋         | 12/189 [00:00<00:12, 13.62it/s]








  7%|▋         | 14/189 [00:01<00:12, 13.50it/s]








  8%|▊         | 16/189 [00:01<00:12, 13.52it/s]








 10%|▉         | 18/189 [00:01<00:13, 12.80it/s]








 11%|█         | 20/189 [00:01<00:15, 11.07it/s]








 12%|█▏        | 22/189 [00:01<00:14, 11.46it/s]








 13%|█▎        | 24/189 [00:01<00:13, 12.13it/s]








 14%|█▍        | 26/189 [

Input Notebook:  Producing channel distributions.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0/Producing channel distributions from GD_AmE_destressed_aligned_w_LTR_CMU_destressed, pc=0.ipynb









  0%|          | 0/189 [00:00<?, ?it/s]








  1%|          | 2/189 [00:00<00:15, 11.77it/s]








  2%|▏         | 4/189 [00:00<00:15, 12.28it/s]








  3%|▎         | 6/189 [00:00<00:14, 12.81it/s]








  4%|▍         | 8/189 [00:00<00:13, 13.08it/s]








  5%|▌         | 10/189 [00:00<00:13, 13.36it/s]








  6%|▋         | 12/189 [00:00<00:13, 13.38it/s]








  7%|▋         | 14/189 [00:01<00:13, 13.19it/s]








  8%|▊         | 16/189 [00:01<00:13, 13.16it/s]








 10%|▉         | 18/189 [00:01<00:13, 13.07it/s]








 11%|█         | 20/189 [00:01<00:15, 10.77it/s]








 12%|█▏        | 22/189 [00:01<00:14, 11.53it/s]








 13%|█▎        | 24/189 [00:01<00:13, 11.86it/s]








 14%|█▍        | 26/189 [00:02<00:1

Input Notebook:  Producing channel distributions.ipynb
Output Notebook: CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/Producing channel distributions from GD_AmE_destressed_aligned_w_LTR_CMU_destressed, pc=0.01.ipynb









  0%|          | 0/189 [00:00<?, ?it/s]








  1%|          | 2/189 [00:00<00:14, 12.92it/s]








  2%|▏         | 3/189 [00:00<00:17, 10.71it/s]








  3%|▎         | 5/189 [00:00<00:15, 11.57it/s]








  4%|▎         | 7/189 [00:00<00:14, 12.22it/s]








  5%|▍         | 9/189 [00:00<00:14, 12.80it/s]








  6%|▌         | 11/189 [00:00<00:13, 13.12it/s]








  7%|▋         | 13/189 [00:00<00:13, 13.33it/s]








  8%|▊         | 15/189 [00:01<00:12, 13.58it/s]








  9%|▉         | 17/189 [00:01<00:12, 13.54it/s]








 10%|█         | 19/189 [00:01<00:14, 11.44it/s]








 11%|█         | 21/189 [00:01<00:14, 11.99it/s]








 12%|█▏        | 23/189 [00:01<00:13, 12.25it/s]








 13%|█▎        | 25/189 [00:01

limit_output extension: Maximum message size of 100000 exceeded with 100001 characters

**How are channel distributions created?**

Take a look at `Producing channel distributions.ipynb`. Besides removing stress information, there are some mathematically non-trivial details that go into defining both uniphone and triphone channel distributions.

## Step 2b: Generating (contextual) lexicon distributions over orthographic vocabularies

Given 
 - a language model $m$ (defined by a `.arpa` file or `kenlm` memory-mapped analogue) 
 - a choice of n-gram contexts $C$ (a `.txt` file with one context per line)
 - a vocabulary $V$ (a `.txt` file with one word per line)
 - a (partial) output filepath $o$ / output filepath prefix $o$
 
`Producing contextual distributions.ipynb` will write a serialized/memory-mapped `numpy` array to $o$.hV_C that defines $-log_2( p(V|C) )$ - slightly transformed output from `kenlm`. It will also write $p(V|C)$ to $o$.pV_C, and copy both $V$ and $C$ to the base directory specified by $o$. (In both cases, each column is associated with the distribution $p(V|c)$ for some $c$.)

In [93]:
contexts
fisher_lm_fps

('buckeye_contexts.txt', 'swbd2003_contexts.txt')

{'lm': 'LM_Fisher/fisher_utterances_main_4gram.mmap',
 'vocab': 'LM_Fisher/fisher_vocabulary_main.txt'}

In [247]:
LDs = [{'LD_dir':'LD_Fisher_vocab_in_Buckeye_contexts',
        'o_fn_stem':'LD_fisher_vocab_in_buckeye_contexts',
        'o':'LD_Fisher_vocab_in_Buckeye_contexts' + '/' + 'LD_fisher_vocab_in_buckeye_contexts',
        'm':fisher_lm_fps['lm'],
        'v':fisher_lm_fps['vocab'],
        'c':buckeye_contexts,
        'nb_fp':path.join('LD_Fisher_vocab_in_Buckeye_contexts', 
                          'Producing ' + 'LD_Fisher_vocab_in_Buckeye_contexts'.replace('_', ' ')[3:] + ' contextual distributions.ipynb')},
       {'LD_dir':'LD_Fisher_vocab_in_swbd2003_contexts',
        'o_fn_stem':'LD_fisher_vocab_in_swbd2003_contexts',
        'o':'LD_Fisher_vocab_in_swbd2003_contexts' + '/' + 'LD_fisher_vocab_in_swbd2003_contexts',
        'm':fisher_lm_fps['lm'],
        'v':fisher_lm_fps['vocab'],
        'c':swbd2003_contexts,
        'nb_fp':path.join('LD_Fisher_vocab_in_swbd2003_contexts', 
                          'Producing ' + 'LD_Fisher_vocab_in_swbd2003_contexts'.replace('_', ' ')[3:] + ' contextual distributions.ipynb')}
      ]

In [248]:
for ld in LDs:
    if not path.exists(ld['LD_dir']):
        mkdir(ld['LD_dir'])
    
    nb = pm.execute_notebook(
    'Producing contextual distributions.ipynb',
#     'Producing ' + ld['LD_dir'].replace('_', ' ')[3:] + ' contextual distributions.ipynb',
    ld['nb_fp'],
    parameters=dict(m = ld['m'],
                    v = ld['v'],
                    c = ld['c'],
                    o = ld['o'])
    )
    print('\n')

Input Notebook:  Producing contextual distributions.ipynb
Output Notebook: LD_Fisher_vocab_in_Buckeye_contexts/Producing Fisher vocab in Buckeye contexts contextual distributions.ipynb










  0%|          | 0/86 [00:00<?, ?it/s]









  3%|▎         | 3/86 [00:00<00:03, 24.86it/s]









  7%|▋         | 6/86 [00:00<00:03, 26.09it/s]









 10%|█         | 9/86 [00:00<00:02, 26.96it/s]









 14%|█▍        | 12/86 [00:00<00:02, 26.84it/s]









 17%|█▋        | 15/86 [00:00<00:02, 26.56it/s]









 21%|██        | 18/86 [00:00<00:02, 25.03it/s]









 24%|██▍       | 21/86 [00:00<00:03, 18.35it/s]









 28%|██▊       | 24/86 [00:01<00:03, 19.58it/s]









 31%|███▏      | 27/86 [00:01<00:02, 20.06it/s]









 35%|███▍      | 30/86 [00:01<00:02, 19.89it/s]









 38%|███▊      | 33/86 [00:01<00:02, 21.35it/s]









 42%|████▏     | 36/86 [00:01<00:02, 21.42it/s]









 45%|████▌     | 39/86 [00:01<00:02, 21.26it/s]









 49%|████▉     | 42

Input Notebook:  Producing contextual distributions.ipynb
Output Notebook: LD_Fisher_vocab_in_swbd2003_contexts/Producing Fisher vocab in swbd2003 contexts contextual distributions.ipynb










  0%|          | 0/87 [00:00<?, ?it/s]









  3%|▎         | 3/87 [00:00<00:03, 24.85it/s]









  8%|▊         | 7/87 [00:00<00:03, 26.28it/s]









 11%|█▏        | 10/87 [00:00<00:02, 26.92it/s]









 15%|█▍        | 13/87 [00:00<00:02, 26.42it/s]









 18%|█▊        | 16/87 [00:00<00:03, 21.61it/s]









 22%|██▏       | 19/87 [00:00<00:03, 22.28it/s]









 25%|██▌       | 22/87 [00:01<00:03, 17.31it/s]









 29%|██▊       | 25/87 [00:01<00:03, 19.44it/s]









 32%|███▏      | 28/87 [00:01<00:03, 17.38it/s]









 34%|███▍      | 30/87 [00:01<00:03, 17.08it/s]









 38%|███▊      | 33/87 [00:01<00:02, 18.75it/s]









 41%|████▏     | 36/87 [00:01<00:02, 19.88it/s]









 45%|████▍     | 39/87 [00:01<00:02, 21.15it/s]









 48%|████▊     |

# Step 3: Creating combinable models

**The basic problem:**

1. **Channel model + transcribed lexicon relation**: Even after the gating data and a transcribed lexicon relation are defined over the same inventory, 
 - the lexicon may contain triphones that are not in the stimuli triphones of a triphone channel model.
 - the channel distribution will contain triphones that cannot be found in the transcribed lexicon relation. (While the other steps here are strictly necessary, this is simply a practical step for making downstream computation faster.)
2. **Language model + transcribed lexicon relation**: The orthographic vocabulary of a transcribed lexicon relation may contain wordforms not in an n-gram model's vocabulary. (We *don't* want to use the out-of-vocabulary estimate for those wordforms.)
3.  **Contextual distributions + transcribed lexicon relation**: The contextual distributions from Step 3b above are defined over the *language model's* orthographic vocabulary, which will likely include wordforms that are not in the transcribed lexicon relation. We want to create modified forms of these distributions where we condition on the choice of an orthographic wordform that is in the transcribed lexicon relation.

Once we have
 - a version $l'$ of the transcribed lexicon relation $l$ trimmed with respect to both the triphones of the channel model $c$ and the (orthographic) vocabulary of a language model $m$
 - a version $d'$ of the contextual distributions over $m$'s vocabulary (with respect to some set of n-gram contexts) $d$ trimmed to only define distributions over $l'$
 - a version $c'$ of the channel model $c$ trimmed with respect to a transcribed lexicon relation $l'$
 - a probability distribution over segmental wordforms given an orthographic wordform
 
we will be able to combine everything together to calculate confusability of wordforms in corpus contexts.

## Step 3a: Filter transcription lexicons to only include words that can be modeled by a given channel distribution

In [ ]:
#gather relevant LTRs and their associated CMs

In [209]:
aligned_LTRs = list(map(lambda ab: {'LTR_fp':ab['my_ol'],
                                    'GD_fp':ab['my_og']},
                        arg_bundles))
list(map(lambda d: d['LTR_fp'],
         aligned_LTRs))

['LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_w_GD_AmE-diphones.tsv',
 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_w_GD_AmE-diphones.tsv',
 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_w_GD_AmE-diphones.tsv']

In [210]:
# CM_dirs = list(map(lambda cab: {'CM_fp':path.join(cab['cm_dir'],'pY1X0X1X2.json'),
#                                 'GD_fp':cab['gating_data_fp']},
#                    filter(lambda cab: cab['c'] != 0, cm_arg_bundles)))
# CM_dirs
# CM_dirs[5]
# # listdir(CM_dirs[0][])

aligned_CMs = list(map(lambda cab: {'CM_fp':path.join(cab['cm_dir'],'pY1X0X1X2.json'),
                                    'GD_fp':cab['gating_data_fp']},
                       filter(lambda cab: cab['c'] != 0 and 'aligned' in cab['gating_data_fp'], 
                              cm_arg_bundles)))
aligned_CMs

[{'CM_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/pY1X0X1X2.json',
  'GD_fp': 'GD_AmE_destressed_aligned_w_LTR_newdic_destressed/GD_AmE-diphones_aligned_w_LTR_newdic_destressed.tsv'},
 {'CM_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/pY1X0X1X2.json',
  'GD_fp': 'GD_AmE_destressed_aligned_w_LTR_newdic_destressed/GD_AmE-diphones_aligned_w_LTR_newdic_destressed.tsv'},
 {'CM_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/pY1X0X1X2.json',
  'GD_fp': 'GD_AmE_destressed_aligned_w_LTR_newdic_destressed/GD_AmE-diphones_aligned_w_LTR_newdic_destressed.tsv'},
 {'CM_fp': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/pY1X0X1X2.json',
  'GD_fp': 'GD_AmE_destressed_aligned_w_LTR_CMU_destressed/GD_AmE-diphones_aligned_w_LTR_CMU_destressed.tsv'},
 {'CM_fp': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/pY1X0X1X2.json',
  'GD_fp': 'GD_AmE_destressed_aligned_w_LTR_CMU_destressed/GD_AmE-diph

In [211]:
def get_aligned_CMs(ltr_bundle):
    matches = [cm_bundle for cm_bundle in aligned_CMs if cm_bundle['GD_fp'] == ltr_bundle['GD_fp']]
    return list(map(lambda d: d['CM_fp'],
                    matches))

In [212]:
aligned_LTRs[0]['LTR_fp']

#NB: all of these will have the same set of stimuli triphones
#    ...which is all we care about here
get_aligned_CMs(aligned_LTRs[0]) 

'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_w_GD_AmE-diphones.tsv'

['CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/pY1X0X1X2.json']

In [213]:
aligned_LTRs_and_CM = [{'LTR_fp':ltr['LTR_fp'],
                        'matching_CMs':get_aligned_CMs(ltr)}
                       for ltr in aligned_LTRs]
aligned_LTRs_and_CM

[{'LTR_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_w_GD_AmE-diphones.tsv',
  'matching_CMs': ['CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/pY1X0X1X2.json',
   'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/pY1X0X1X2.json',
   'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/pY1X0X1X2.json']},
 {'LTR_fp': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_w_GD_AmE-diphones.tsv',
  'matching_CMs': ['CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/pY1X0X1X2.json',
   'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/pY1X0X1X2.json',
   'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.1/pY1X0X1X2.json']},
 {'LTR_fp': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_w_GD_AmE-diphones.tsv',
  'matching_CMs': ['CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/pY1X0X1X2.json',
   'CM_AmE_destressed_aligned_w_LTR

In [214]:
for each in aligned_LTRs_and_CM:
    each['c'] = each['matching_CMs'][0]
    each['l'] = each['LTR_fp']
    o_dir = path.dirname(each['LTR_fp'])
    o_fn = path.basename(each['LTR_fp']).split('w_')[0] + 'CM_filtered' + '.tsv'
    each['o'] = path.join(o_dir, o_fn)
    
    nb_fn = 'Filter ' + o_fn.split('_aligned')[0] + ' against channel model.ipynb'
    each['nb_fp'] = path.join(o_dir, nb_fn)
    
    print('c = {0}\nl = {1}\no = {2}\nnb = {3}'.format(each['c'], each['l'], each['o'], each['nb_fp']))
    print(' ')

c = CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/pY1X0X1X2.json
l = LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_w_GD_AmE-diphones.tsv
o = LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered.tsv
nb = LTR_newdic_destressed_aligned_w_GD_AmE_destressed/Filter LTR_newdic_destressed against channel model.ipynb
 
c = CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/pY1X0X1X2.json
l = LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_w_GD_AmE-diphones.tsv
o = LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered.tsv
nb = LTR_CMU_destressed_aligned_w_GD_AmE_destressed/Filter LTR_CMU_destressed against channel model.ipynb
 
c = CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/pY1X0X1X2.json
l = LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_w_GD_AmE-diphones.tsv
o = LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Bu

In [219]:
for each in aligned_LTRs_and_CM:
    if not path.exists(path.dirname(each['o'])):
        mkdir(path.dirname(each['o']))
    
    nb = pm.execute_notebook(
    'Filter transcription lexicon by channel model.ipynb',
    each['nb_fp'],
    parameters=dict(l = each['l'],
                    c = each['c'],
                    o = each['o'])
    )
    print('\n')

Input Notebook:  Filter transcription lexicon by channel model.ipynb
Output Notebook: LTR_newdic_destressed_aligned_w_GD_AmE_destressed/Filter LTR_newdic_destressed against channel model.ipynb









  0%|          | 0/27 [00:00<?, ?it/s]








 22%|██▏       | 6/27 [00:00<00:00, 57.63it/s]








 52%|█████▏    | 14/27 [00:00<00:00, 60.72it/s]








 67%|██████▋   | 18/27 [00:01<00:00,  9.16it/s]








 89%|████████▉ | 24/27 [00:01<00:00, 12.15it/s]








100%|██████████| 27/27 [00:02<00:00, 11.95it/s]

Input Notebook:  Filter transcription lexicon by channel model.ipynb
Output Notebook: LTR_CMU_destressed_aligned_w_GD_AmE_destressed/Filter LTR_CMU_destressed against channel model.ipynb









  0%|          | 0/27 [00:00<?, ?it/s]








 33%|███▎      | 9/27 [00:00<00:00, 88.75it/s]








 59%|█████▉    | 16/27 [00:00<00:00, 29.56it/s]








 70%|███████   | 19/27 [00:01<00:01,  6.93it/s]








 89%|████████▉ | 24/27 [00:02<00:00,  9.33it/s]








100%|██████████| 27/27 [00:03<00:00,  4.32it/s]

Input Notebook:  Filter transcription lexicon by channel model.ipynb
Output Notebook: LTR_Buckeye_aligned_w_GD_AmE_destressed/Filter LTR_Buckeye against channel model.ipynb









  0%|          | 0/27 [00:00<?, ?it/s]








  7%|▋         | 2/27 [00:00<00:01, 19.22it/s]








 41%|████      | 11/27 [00:00<00:00, 25.02it/s]








 59%|█████▉    | 16/27 [00:00<00:00, 28.23it/s]








 74%|███████▍  | 20/27 [00:01<00:00,  8.63it/s]








 93%|█████████▎| 25/27 [00:01<00:00, 10.99it/s]








100%|██████████| 27/27 [00:01<00:00, 15.32it/s]

## Step 3b: Filter transcription lexicons to only include words that are in a language model's vocabulary

**NB** For this step, we will use shell commands provided by the `csvkit` package.

In [170]:
fisher_lm_fps
fisher_lm_vocab_fp

{'lm': 'LM_Fisher/fisher_utterances_main_4gram.mmap',
 'vocab': 'LM_Fisher/fisher_vocabulary_main.txt'}

'LM_Fisher/fisher_vocabulary_main.txt'

In [179]:
LTR_fps = list(map(lambda pair: pair['LTR_fp'],
                   aligned_LTRs))
LTR_fps

LTR_CM_filtered = list(map(lambda d: d['o'],
                           aligned_LTRs_and_CM))
LTR_CM_filtered

['LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_w_GD_AmE-diphones.tsv',
 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_w_GD_AmE-diphones.tsv',
 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_w_GD_AmE-diphones.tsv']

['LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered.tsv',
 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered.tsv',
 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered.tsv']

How many lexicon entries have unmodelable triphones? (We'll next check how many such lexicon entries aren't in the language model vocabulary.)

In [220]:
!wc -l LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_w_GD_AmE-diphones.tsv
!wc -l LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_w_GD_AmE-diphones.tsv
!wc -l LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_w_GD_AmE-diphones.tsv

19529 LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_w_GD_AmE-diphones.tsv
133855 LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_w_GD_AmE-diphones.tsv
7999 LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_w_GD_AmE-diphones.tsv


In [221]:
!wc -l LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered.tsv
!wc -l LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered.tsv
!wc -l LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered.tsv

17079 LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered.tsv
127799 LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered.tsv
7011 LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered.tsv


We'll be using `csvjoin` to do an inner join of the LM vocabulary with each transcribed lexicon relation:
```
csvjoin -t LTR_LEXNAME.tsv LM_VOCAB.tsv -c Orthographic_Wordform | csvformat -T > LTR_LEXNAME_filtered.tsv
```
Performs an inner join of the transcribed lexicon relation in `LTR_LEXNAME.tsv` with the language model vocabulary in `LM_VOCAB.tsv` along the `Orthographic_Wordform` column and writes the result to `LTR_LEXNAME_filtered.tsv`. `LTR_LEXNAME_filtered.tsv` will only contain those rows of `LTR_LEXNAME.tsv` where the `Orthographic_Wordform` value is contained in `LM_VOCAB.tsv`.

To do this, we first need to label the vocabulary file with the right column name:

In [222]:
!cat -n LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered.tsv | head -10
!cat -n LM_Fisher/fisher_vocabulary_main.txt | head -10

     1	Orthographic_Wordform	Transcription
     2	a	ə
     3	aardvark	ɑ.ɹ.d.v.ɑ.ɹ.k
     4	aback	ə.b.æ.k
     5	abacus	æ.b.ə.k.ə.s
     6	abaft	ə.b.æ.f.t
     7	abalone	æ.b.ə.l.oʊ.n.i
     8	abandon	ə.b.æ.n.d.ɪ.n
     9	abase	ə.b.eɪ.s
    10	abash	ə.b.æ.ʃ
cat: write error: Broken pipe
     1	'and
     2	'berserkly'
     3	'bout
     4	'burb
     5	'burban
     6	'burbs
     7	'cau
     8	'cause
     9	'cept
    10	'cide
cat: write error: Broken pipe


In [223]:
!echo "Orthographic_Wordform\n$(cat LM_Fisher/fisher_vocabulary_main.txt)" > LM_Fisher/fisher_vocabulary_main.tsv
!cat -n LM_Fisher/fisher_vocabulary_main.tsv | head -10

     1	Orthographic_Wordform
     2	'and
     3	'berserkly'
     4	'bout
     5	'burb
     6	'burban
     7	'burbs
     8	'cau
     9	'cause
    10	'cept
cat: write error: Broken pipe


**A bit less than half** of the triphone channel model-modelable `newdic` lexicon **isn't** in the LM vocab:

In [224]:
!csvjoin -t LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered.tsv LM_Fisher/fisher_vocabulary_main.tsv -c Orthographic_Wordform | csvformat -T > LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv
!wc -l LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered.tsv
!wc -l LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv

17079 LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered.tsv
9416 LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv


**About three quarters** of the triphone channel model-modelable `CMU_destressed` lexicon **isn't** in the LM vocab:

In [226]:
!csvjoin -t LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered.tsv LM_Fisher/fisher_vocabulary_main.tsv -c Orthographic_Wordform | csvformat -T > LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv
!wc -l LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered.tsv
!wc -l LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv

127799 LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered.tsv
33120 LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv


**Less than 10%** of the triphone channel model-modelable `Buckeye` lexicon **isn't** in the LM vocab:

In [225]:
!csvjoin -t LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered.tsv LM_Fisher/fisher_vocabulary_main.tsv -c Orthographic_Wordform | csvformat -T > LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv
!wc -l LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered.tsv
!wc -l LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv

7011 LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered.tsv
6574 LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv


Collecting the filtered transcribed lexicon relations...

In [228]:
LTR_CM_filtered_LM_filtered = list(map(lambda fp: fp[:-4] + '_LM_filtered.tsv',
                                       LTR_CM_filtered))
LTR_CM_filtered_LM_filtered

['LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv']

## Step 3c: Filter the conditioning events of channel distributions to only include triphones contained in a transcription lexicon's segmental wordform list

## Step 3d: For each (filtered) transcribed lexicon relation, define the relevant contextual lexicon distributions over orthographic wordforms

## Step 3e: For each (filtered) transcribed lexicon relation, define a conditional distribution on segmental wordforms given an orthographic wordform

In [230]:
LTR_CM_filtered_LM_filtered

['LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv']

In [242]:
pW_V_fp_bundles = []
for ltr_fp in LTR_CM_filtered_LM_filtered:
    LTR_n = path.basename( ltr_fp )[:-4]
    LTR_dir = path.dirname( ltr_fp )
    pW_V_fp_bundles.append({'LTR_fp':ltr_fp,
                          'pW_V_fp':ltr_fp[:-4] + '.pW_V.json',
                          'nb_fp':path.join(LTR_dir,'Define pW_V given {0}.ipynb'.format(LTR_n))})
pW_V_fp_bundles

[{'LTR_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv',
  'pW_V_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.pW_V.json',
  'nb_fp': 'Define pW_V given LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.ipynb'},
 {'LTR_fp': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv',
  'pW_V_fp': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.pW_V.json',
  'nb_fp': 'Define pW_V given LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.ipynb'},
 {'LTR_fp': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv',
  'pW_V_fp': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.pW_V.json',
  'nb_fp': 'Define pW_V given LTR_Buckeye_aligned_CM_filtered_LM_filtered.ipynb'}]

In [243]:
for each in pW_V_fp_bundles:
    pW_V_fp_output_dir = path.dirname(each['pW_V_fp'])
    if not path.exists(pW_V_fp_output_dir):
        mkdir(pW_V_fp_output_dir)
    
    nb = pm.execute_notebook(
    'Define a conditional distribution on segmental wordforms given an orthographic one.ipynb',
    each['nb_fp'],
    parameters=dict(l = each['LTR_fp'],
                    o = each['pW_V_fp'])
    )
    print('\n')

{'LTR_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv', 'pW_V_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.pW_V.json', 'nb_fp': 'Define pW_V given LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.ipynb'}


Input Notebook:  Define a conditional distribution on segmental wordforms given an orthographic one.ipynb
Output Notebook: Define pW_V given LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.ipynb









  0%|          | 0/34 [00:00<?, ?it/s]








 18%|█▊        | 6/34 [00:00<00:00, 57.83it/s]








 38%|███▊      | 13/34 [00:00<00:00, 60.05it/s]








 50%|█████     | 17/34 [00:00<00:00, 33.00it/s]








 59%|█████▉    | 20/34 [00:00<00:00, 31.34it/s]








 74%|███████▎  | 25/34 [00:00<00:00, 35.23it/s]








 85%|████████▌ | 29/34 [00:01<00:00,  9.01it/s]








 94%|█████████▍| 32/34 [00:02<00:00, 10.10it/s]








100%|██████████| 34/34 [00:02<00:00, 15.88it/s]



{'LTR_fp': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv', 'pW_V_fp': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.pW_V.json', 'nb_fp': 'Define pW_V given LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.ipynb'}


Input Notebook:  Define a conditional distribution on segmental wordforms given an orthographic one.ipynb
Output Notebook: Define pW_V given LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.ipynb









  0%|          | 0/34 [00:00<?, ?it/s]








 21%|██        | 7/34 [00:00<00:00, 68.96it/s]








 38%|███▊      | 13/34 [00:00<00:00, 65.73it/s]








 50%|█████     | 17/34 [00:00<00:00, 38.49it/s]








 59%|█████▉    | 20/34 [00:00<00:00, 27.81it/s]








 71%|███████   | 24/34 [00:00<00:00, 30.56it/s]








 79%|███████▉  | 27/34 [00:11<00:07,  1.11s/it]








 85%|████████▌ | 29/34 [00:12<00:04,  1.04it/s]








 91%|█████████ | 31/34 [00:13<00:02,  1.33it/s]








100%|██████████| 34/34 [00:13<00:00,  2.54it/s]



{'LTR_fp': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv', 'pW_V_fp': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.pW_V.json', 'nb_fp': 'Define pW_V given LTR_Buckeye_aligned_CM_filtered_LM_filtered.ipynb'}


Input Notebook:  Define a conditional distribution on segmental wordforms given an orthographic one.ipynb
Output Notebook: Define pW_V given LTR_Buckeye_aligned_CM_filtered_LM_filtered.ipynb









  0%|          | 0/34 [00:00<?, ?it/s]








 21%|██        | 7/34 [00:00<00:00, 63.71it/s]








 41%|████      | 14/34 [00:00<00:00, 65.44it/s]








 53%|█████▎    | 18/34 [00:00<00:00, 37.04it/s]








 65%|██████▍   | 22/34 [00:00<00:00, 36.41it/s]








 76%|███████▋  | 26/34 [00:01<00:00, 14.93it/s]








 85%|████████▌ | 29/34 [00:01<00:00, 13.27it/s]








 91%|█████████ | 31/34 [00:01<00:00, 13.11it/s]








100%|██████████| 34/34 [00:01<00:00, 20.17it/s]

# Step 4: Perform forward model calculations

## Step 4a: Define an incremental channel distribution on segmental wordforms 

## Step 4b: Define a channel distribution on (whole) orthographic wordforms

# Step 5: Generating posterior distributions

# Step 6: Generating analysis measures